# Previsão de Estoque para 28 dias

In [ ]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

# Google Cloud Platform
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import bigquery

from datetime import timedelta

# Caminho para o arquivo de chave da conta de serviço
service_account_path = 'tfm-sa.json'

# Criar objeto de credenciais
credentials = service_account.Credentials.from_service_account_file(service_account_path)

# Configurações do projeto e dataset
project_id = 'perseverance-332400'
dataset_id = 'TFM'
table_id = 'ds_market_forecast'  # Nova tabela para armazenar as previsões
full_table_id = f'{project_id}.{dataset_id}.ds_market'  # Tabela original com os dados

# Função para configurar o cliente BigQuery usando credenciais específicas
def initialize_bigquery_client():
    return bigquery.Client(project=project_id, credentials=credentials)

# Função para extrair dados históricos do BigQuery
def get_historical_data_from_bigquery(query: str, client) -> pd.DataFrame:
    query_job = client.query(query)
    data = query_job.to_dataframe()
    print(f"Dados extraídos do BigQuery com {data.shape[0]} linhas e {data.shape[1]} colunas.")
    return data

# Função para preparar os dados
def prepare_sales_data(raw_data: pd.DataFrame) -> pd.DataFrame:
    # Converter coluna de data
    raw_data['date'] = pd.to_datetime(raw_data['date'])
    # Preencher valores ausentes na coluna de vendas
    raw_data['sales'] = raw_data['sales'].fillna(0)
    # Agrupar por data e somar as vendas
    sales_data = raw_data.groupby('date').agg({'sales': 'sum'}).sort_index()
    print("Dados de vendas preparados e limpos.")
    return sales_data

# Função para treinar o modelo ARIMA
def train_forecast_model(data: pd.DataFrame, order=(1, 1, 1)):
    print("Treinando o modelo ARIMA...")
    model = ARIMA(data['sales'], order=order)
    model_fit = model.fit()
    print("Modelo treinado com sucesso.")
    return model_fit

# Função para gerar previsões
def predict_sales(data: pd.DataFrame, model, steps=30) -> pd.DataFrame:
    print(f"Gerando previsões para {steps} dias...")
    forecast = model.get_forecast(steps=steps)
    forecast_df = forecast.summary_frame()
    # Criar uma coluna de datas para as previsões
    forecast_df['date'] = pd.date_range(data.index[-1] + timedelta(days=1), periods=steps)
    forecast_df = forecast_df.reset_index(drop=True)
    forecast_df = forecast_df[['date', 'mean']]
    forecast_df.columns = ['date', 'forecast_sales']
    print("Previsão gerada com sucesso.")
    return forecast_df

# Função para carregar previsões no BigQuery
def store_forecast_results(data: pd.DataFrame, table_id: str, client):
    print(f"Carregando previsões para a tabela {table_id} no BigQuery...")
    job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")
    job = client.load_table_from_dataframe(data, table_id, job_config=job_config)
    job.result()  # Espera o job terminar
    print("Previsões carregadas com sucesso no BigQuery.")

# Pipeline completo para previsão de estoque dos top 10 itens por loja
def run_forecast_pipeline_for_top_items_per_store(query: str, table_id: str, steps=30, order=(1, 1, 1)):
    client = initialize_bigquery_client()
    
    # Extrair dados históricos do BigQuery
    raw_data = get_historical_data_from_bigquery(query, client)
    
    # Converter coluna de data
    raw_data['date'] = pd.to_datetime(raw_data['date'])
    
    # Verificar se as colunas necessárias existem
    required_columns = {'store', 'item', 'date', 'sales'}
    if not required_columns.issubset(raw_data.columns):
        missing = required_columns - set(raw_data.columns)
        raise ValueError(f"As colunas a seguir estão faltando nos dados: {missing}")
    
    # Obter lista de lojas únicas
    stores = raw_data['store'].unique()
    
    all_forecasts = []
    
    for store in stores:
        print(f"\nProcessando loja {store}...")
        store_data = raw_data[raw_data['store'] == store]
        
        # Calcular vendas totais por item nesta loja
        total_sales_per_item = store_data.groupby('item')['sales'].sum().reset_index()
        
        # Identificar os top 10 itens mais vendidos nesta loja
        top_items = total_sales_per_item.sort_values(by='sales', ascending=False).head(10)['item']
        
        for item in top_items:
            print(f"Processando item {item} na loja {store}...")
            item_data = store_data[store_data['item'] == item]
            # Preparar dados
            prepared_data = prepare_sales_data(item_data)
            
            # Verificar se há dados suficientes para treinamento
            if len(prepared_data) < 2:
                print(f"Dados insuficientes para item {item} na loja {store}, pulando...")
                continue
            
            try:
                # Treinar modelo
                model = train_forecast_model(prepared_data, order=order)
                # Fazer previsão
                forecast = predict_sales(prepared_data, model, steps=steps)
                forecast['store'] = store
                forecast['item'] = item
                all_forecasts.append(forecast)
            except Exception as e:
                print(f"Erro ao processar item {item} na loja {store}: {e}")
                continue
    
    if all_forecasts:
        # Concatenar todos os resultados
        final_forecast = pd.concat(all_forecasts, ignore_index=True)
        
        # Carregar previsões no BigQuery
        store_forecast_results(final_forecast, table_id, client)
    else:
        print("Nenhuma previsão foi gerada.")

# Definir a consulta SQL para carregar os dados de 2015
query = f"""
SELECT *
FROM {full_table_id}
WHERE EXTRACT(YEAR FROM date) = 2015
"""

# Executar o pipeline
run_forecast_pipeline_for_top_items_per_store(
    query,
    f'{project_id}.{dataset_id}.{table_id}',
    steps=30,
    order=(1, 1, 1)
)


c:\Users\rpiaz\Desenvolvimento\DSMarket\.venv\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Dados extraídos do BigQuery com 58354236 linhas e 14 colunas.
